## Please don't run / don't click "Run all" the notebook:

At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.

# PPE Safety Compliance in Manufacturing - Hard Hat Detection  
<h3><span style="color: #117d30;"> Using Azure Custom Vision Congnitive Services</span></h3>

## Scenario Overview

**Azure Custom Vision** is a cognitive service which helps in building and deploying object detection and image classification models. 

This notebook provides an end-to-end demo on how to use the Azure Cognitive Services Custom Vision API to build and deploy an object detection model to detect **PPE compliance related to hard hats**. 

In this scenario we will see how the model provides the count of all workers and the count of workers wearing hard hats. All incidents of PPE non-compliance, where any worker is not wearing a hard hat, are marked as Non-Compliant. 



+ This demo uses real-world video footage from a manufacturing setting 
+ This raw video is stored in an ADLSGen2 storage container  
+ In this notebook, we process this raw video using Azure Custom Vision Cognitive Services model and get a processed video complete with: 

    * Moving Bounding boxes and moving tags for workers
    * Moving Bounding boxes and moving tags for hard hats
    * Count of workers and hard hats
    * Frame-wise annotation of Compliance and Non-Compliance 

## Setting-Up: 

+ Initiating Spark Context 
+ Defining the storage, file path and file name for **raw input** video 
+ Defining the storage, file path and file name for **processed output** video

In [3]:
%%pyspark

## setting up the credentials for ADLS Gen2
import os
from pyspark.sql import SparkSession
from azure.storage.blob import BlockBlobService

# setting up blob storage configs
STORAGE_ACCOUNT_NAME = "#DATA_LAKE_NAME#"
STORAGE_ACCOUNT_ACCESS_KEY = "#STORAGE_ACCOUNT_KEY#"
STORAGE_CONTAINER_NAME = "ppecompliancedetection"

# setting-up spark context
session = SparkSession.builder.getOrCreate() 
session.conf. \
 set("fs.azure.account.key.{}.blob.core.windows.net".format(STORAGE_ACCOUNT_NAME), \
 STORAGE_ACCOUNT_ACCESS_KEY) 

# defining the folders for raw input video and processed output video  
input_folder="HardHat/Input/"
output_folder="HardHat/Output/"

# getting the file name from the folder in blob folder
blob_service = BlockBlobService(STORAGE_ACCOUNT_NAME, STORAGE_ACCOUNT_ACCESS_KEY) 

# providing the filename from input folder
input_file="threemenworkers.mp4"
output_file="processed_threemenworkers.mp4"
blob_file=input_folder + input_file

# downloading the file from the blob storage
download_file_path = os.path.join(os.getcwd(), input_file)
blob_service.get_blob_to_path(STORAGE_CONTAINER_NAME, blob_file ,download_file_path)

# defining the input and output video paths 
input_video_path=download_file_path
output_video_path=os.path.join(os.getcwd(), "Output",output_file)



## Installing OpenCV and importing other python libraries  

OpenCV is one of the most popular computer vision libraries for image and video processing. In this notebook, we will install python binding for  OpenCV. 

We will also import other libraries like  **matplotlib**,  **numpy**, **pandas**, etc. that are required at different stages in this notebook for manipulating and processing images.    

In [4]:
%%pyspark
import os

# installing and importing the OpenCV library 
# openCV will be used to grab frames from video 
# and for converting the frames into arrays   
import pip
pip.main(["install","opencv-python-headless"])
pip.main(["install","grequests"])
import cv2 

# and for processing arrays   
import numpy as np

# for encoding and decoding Custom Vision predictions 
import json

# for converting the Custom Vision predictions to dataframe   
import pandas as pd

# making web service calls in batches
import grequests 

# for file name pattern matching   
import fnmatch  

# for displaying images from the processes output video   
import matplotlib.pyplot as plt

# importing other required libraries
import random
import textwrap
import datetime 
from PIL import Image
import time 

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

## Making API call to Custom Vision Cognitive Service  

Now we will define connections details for making the API call to the Custom Vision Service 


In [5]:
%%pyspark

# web service end-point for the Custom Vision model    
# we will process video frames (which are images)   
POST_URL = "https://westus2.api.cognitive.microsoft.com/customvision/v3.0/Prediction/b2e4f4ce-d9f1-4fb7-aa0c-2f50fdc14d1b/detect/iterations/Iteration4/image"

# providing prediction key
HEADERS = {'Prediction-Key': "7f743d4b8d6d459fb2bb0e8648dfa38e","Content-Type":"application/json"}

# number of API calls per pool of request   
MAX_CONNECTIONS = 100 

# initializing the height and width for frames in the video 
WIDTH = 0
HEIGHT = 0

## Functions to convert raw input video to processed video   
  

### function: getFrames

This function takes a  video as input and returns list of frames    

In [6]:
%%pyspark
# returns frame array of video
def getFrames(path):
    images = []
    byteImages = []
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            images.append(image)
            count += 1
    global HEIGHT, WIDTH
    HEIGHT, WIDTH, lay = images[0].shape
    return images

### function: convertCVImagesToByteImages

This function takes a list of frames as input and returns list of Byte arrays   

In [7]:
%%pyspark
# Returns ByteImages for POST request params
def convertCVImagesToByteImages(frames):
    toReturnByteImages = []
    for frame in frames:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        is_success, im_buf_arr = cv2.imencode(".jpg", image)
        if is_success:
            byte_im = im_buf_arr.tobytes()
            toReturnByteImages.append(byte_im)
    return toReturnByteImages

### function: makeRequests 

This function takes a list of ByteImages and a prediction threshold as inputs and returns a list of bounding box co-ordinates, a list of tag data, and a list of Custom Vision service responses in data frames. The function calls some other functions that are also defined in this cell.    

In [8]:
%%pyspark 

# defining a function for making batch HTTP requests
def batchRequest(byteImages):
    results = []
    for x in range(0,len(byteImages)+1, MAX_CONNECTIONS):
        rs = (grequests.post(POST_URL,headers=HEADERS,data=u,stream=False) for u in byteImages[x:x+MAX_CONNECTIONS])
        time.sleep(0.2)
        results.extend(grequests.map(rs)) 
    response_list=[response.text for response in results]
    return response_list

# defining a function to convert raw JSON to a dataframe
def getDFfromResponse(response,threshold):
    resp = json.loads(response)
    pred= resp['predictions']
    pred_df=pd.DataFrame(pred)
    resp_df = pred_df[pred_df['probability'] > float(threshold)]
    return resp_df

# defining a function to converting the bounding box dataframe to a list 
def boundingBoxListFromDF(df):
    cood_list = df['boundingBox'].tolist()
    return cood_list

# defining a function for adjusting the size of bounding box 
def fitAccordingToSize(coods):
    print("Multiplying by " + str(WIDTH) + " , " + str(HEIGHT))
    for i in range(len(coods)):
        coods[i]['left']=coods[i]['left']*WIDTH
        coods[i]['top']=coods[i]['top']*HEIGHT
        coods[i]['width']=coods[i]['width']*WIDTH
        coods[i]['height']=coods[i]['height']*WIDTH
    return coods


#==================== function: makeRequests ====================#  
# defining a function to get bounding box coordinates and tags
# makes calls to other functions defined above 
def makeRequests(byteImages,threshold):
    tagsData = []
    bound_box_cood = []
    responseJSON =  []
    response_array=batchRequest(byteImages)
    for i,response in enumerate(response_array):
        df = getDFfromResponse(response, threshold)
        listBx = boundingBoxListFromDF(df)
        listBx = fitAccordingToSize(listBx)
        taglist = df['tagName'].tolist()
        tagsData.append(taglist)
        bound_box_cood.append(listBx)
        responseJSON.append(df)
    return bound_box_cood,tagsData,responseJSON 

### function: isCompliant 

This function takes a list of tags for a given frame as input and returns a Boolean after matching the count of worker tags and helmet tags.  

In [9]:
%%pyspark
# defining a function to check compliance or non-compliance for a frame
def isCompliant(tags):
    return (len(list(filter(lambda x: x == 'helmet', tags))) == len(list(filter(lambda x: x == 'worker',tags)))) 


### function: printCNC

This function checks if a frame is compliant or non-compliant and then adds the appropriate labels. 

In [10]:
%%pyspark

# Print Compliance Label
def PrintCompliance(frames,i,font,lineType):
    frames[i] = cv2.rectangle(
        frames[i], (WIDTH - 300, 0), (WIDTH, 50), (0, 128, 0), -1)
    frames[i] = cv2.putText(frames[i], "COMPLIANT",
                            (WIDTH - 220, 33),
                            font,
                            1,
                            (255, 255, 255), 2,
                            lineType)

# Print Non-Compliance Label
def PrintNonCompliance(frames, i, font, lineType):
    frames[i] = cv2.rectangle(
        frames[i], (WIDTH - 300, 0), (WIDTH, 50), (0, 0, 128), -1)
    frames[i] = cv2.putText(frames[i], "NON-COMPLIANT",
                            (WIDTH - 275, 34),
                            font,
                            1,
                            (255, 255, 255), 2,
                            lineType)  

#==================== function: printCNC ====================#  
# Check for compliance or non-compliance label
def printCNC(which, frames, i, font, lineType):
    if which:
        PrintCompliance(frames, i, font, lineType)
    else:
        PrintNonCompliance(frames, i, font, lineType)                                                      

### function: SaveVideo

We are using this function to convert a list of frames into a video.  


In [11]:
%%pyspark

# defining a function to convert a list of frames into a video 
def SaveVideo(frames, output_file_name):
    img_array = []
    for img in frames:
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(output_file_name,cv2.VideoWriter_fourcc(*'MP4V'), 15, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

## function: ConvertVideo 

This function combines raw frames into a processed video. It relies on several other functions that have been defined above. 

In [12]:
%%pyspark

# defining a function to convert raw input video to processed video with tags and stats
def ConvertVideo(video_path, output_file_name, threshold, nframes):
    frames = getFrames(video_path) # this converts the input video into frames   
    
    if nframes != 'all':
        frames = frames[:int(nframes)]
    
    byteImages = convertCVImagesToByteImages(frames) # this converts frames into list of byte images
    
    coodsData,tagsData, responses = makeRequests(byteImages, threshold) # this returns lists of coordinate data, tag data, responses in dataframe   
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (50,50)
    fontScale              = 0.5
    fontColor              = (255,255,255)
    lineType               = 1
    frames_stats = []

    for i,coods_single in enumerate(coodsData):
        dflist = responses[i].drop('boundingBox', axis=1)
        print(dflist['tagId'])
        dflist['tagId'] = (dflist['tagId'].to_string())[-10:]
        dflist.reset_index(drop=True, inplace=True)
        d = dflist.to_dict(orient='records')
        jo = json.dumps(d)
        width = 600
        height = 1080
        blank_image=np.zeros((height, width, 3), np.uint8)
        n_helmets = len([x for x in tagsData[i] if x == 'helmet'])
        n_workers = len([x for x in tagsData[i] if x == 'worker'])
        color = (0, 0, 0)
        blank_image[:height//2] = color
        blank_image[height//2:] = (0, 0, 0)
        blank_image = cv2.putText(blank_image, "STATISTICS" ,(230, 70),font, 1,(66, 245,173),2 ,lineType)
        blank_image = cv2.rectangle(blank_image, (80,150), (width-60, 220), (66, 245,173), 2)
        blank_image = cv2.rectangle(blank_image, (80, 260), (width-60, 330), (66, 245, 173), 2)
        blank_image = cv2.putText(blank_image, "NO OF WORKERS:         "+ str(n_workers),(90+20, 195),font, 0.9,(66, 245,173),2 ,lineType)
        blank_image = cv2.putText(blank_image, "NO OF HELMETS:          "+ str(n_helmets),(90+20, 305),font, 0.9,(66, 245,173),2 ,lineType)
        relative_start = height//2 + - 150
        x, y0 = (50, relative_start)
        text = dflist.to_string()
        thickness = 10
        text_size, _ = cv2.getTextSize(text, font, fontScale, thickness)
        line_height = text_size[1] + 20
    
        if len(text) != 0:
            for k, line in enumerate(text.split("\n")):
                y = y0 + k * line_height
                relative_start = y
                if k == 0:
                    ctext = '      Probability      Id          Tag'
                    blank_image = cv2.putText(blank_image, ctext, (x, y), font, 0.8, (66, 245,173),2, lineType)
                else:
                    blank_image = cv2.putText(blank_image, line, (x, y), font, 0.8, (66, 245,173),2, lineType)
   
        wrapped_text = textwrap.wrap(jo, width=50)
        x, y0 = (10, relative_start)
    
        if relative_start > 700:
            y = relative_start
        else:
            y = 700
    
        for p, line in enumerate(wrapped_text):
            print(line)
            y = y + 30
            blank_image =  cv2.putText(blank_image, line, (10, y), font, 0.7, (66, 245,173),2, lineType)

        frames_stats.append(blank_image)
        last = None

        if len(tagsData[i]) == 0:
            pass

        elif not isCompliant(tagsData[i]): # this checks for compliance for the frame
            if i % 5 == 0:
                PrintNonCompliance(frames, i, font, lineType) # this prints non-compliant labels   
                last = 0
            else:
                if last == None:
                    PrintNonCompliance(frames, i, font, lineType) 
                    last = 0
                else:
                    printCNC(last, frames, font, lineType) # this checks for compliance before applying compliance labels

        else:
            if i % 5 == 0:
                PrintCompliance(frames, i, font, lineType) # this prints compliant labels 
                last = 1
            else:
                if last == None:
                    PrintCompliance(frames, i, font, lineType)
                    last = 1
                else:
                    printCNC(last, frames, font, lineType)

        for j,single in enumerate(coods_single):
            # this puts the tab on top of the detected item
            frames[i] = cv2.rectangle(frames[i],  (int(single['left']) - 3, int(single['top']) - 30), ((int(single['left']) + 100, int(single['top']) - 3)), (66, 245,173), -1)
            frames[i] = cv2.putText(frames[i],tagsData[i][j],
                (int(single['left'] + 2), int(single['top']) + - 8),
                font,
                0.9,
                (0,0,0),2,
                lineType)

            # this creates the bounding box
            frames[i] = cv2.rectangle(frames[i],  (int(single['left']), int(single['top'])), (int(single['left'])+int(single['width']), int(single['top'])+int(single['height'])), (66, 245,173) ,3)
       
        frames[i] = np.concatenate((frames[i], frames_stats[i]), axis=1)

    SaveVideo(frames, output_file_name) # this converts frames into a video
    stats_path = output_file_name.replace('Output', 'Stats')
    SaveVideo(frames_stats, stats_path)

## Video conversion and ADLSGen2 Storage  

We will make functions calls to convert the raw input video to processed video and store the processed video to ADLSGen2 storage. 

In [13]:
%%pyspark
# creating Output and Stats directories for saving processed videos 
if not os.path.isdir("Output"):
   os.mkdir("Output")
if not os.path.isdir("Stats"):
   os.mkdir("Stats")

In [14]:
%%pyspark
# converting raw input video to processed video complete with tags and stats
threshold=0.7
nframes="all"
ConvertVideo(input_video_path, output_video_path,threshold, nframes)

Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
Multiplying by 1920 , 1080
M

In [15]:
%%pyspark
%%capture

# uploading the processed video to the Output directory in the ADLSGen2 storage container
output_blob_file= output_folder + output_file
blob_service.create_blob_from_path(STORAGE_CONTAINER_NAME,output_blob_file ,output_video_path)

## Displaying frames from processed video


In [16]:
%%pyspark 

# saving processed video as images inside the frames directory
images = []
byteImages = []
vidObj = cv2.VideoCapture(output_video_path)
count = 0
success = 1
currentDir = os.getcwd()
if not os.path.isdir("frames"):
   os.mkdir("frames")
while success:
      success, image = vidObj.read()
      if success:
          cv2.imwrite("frames/frame%d.jpg" % count, image) 
          count += 1

images_num=(len(fnmatch.filter(os.listdir(os.getcwd() + "/frames"), '*.jpg')))

# displaying images from frames directory
i=0
for i in range(images_num):
    if i%75==0:
      a = plt.imread("frames/frame%d.jpg" % i)
      plt.imshow(a)
      plt.show()